<a href="https://colab.research.google.com/github/LucasFortolan/ProgramacaoLinear/blob/main/Codigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

In [ ]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, value, LpStatus

# --- DADOS FIXOS DO MODELO (Necessidades Ideais de Nutrientes por Hectare, em kg/ha) ---
# Fonte: Dados fictícios baseados em necessidades típicas.
NECESSIDADES_IDEAIS = {
    "Arroz":  {"N": 30,  "P": 60,  "K": 90},  # kg/ha
    "Feijao": {"N": 192, "P": 100, "K": 60},
    "Milho":  {"N": 35,  "P": 80,  "K": 70},
}

# --- 1. ENTRADA DE DADOS DO USUÁRIO (Tornando o modelo interativo) ---
print("--- 📥 Entrada de Dados do Usuário ---")

# Níveis Atuais do Solo (Assumindo que são medidos para a área total)
print("----- Níveis Atuais de Nutrientes no Solo (em kg/ha) -----")
n_atual = float(input("Digite o Nível ATUAL de Nitrogênio (N) no solo (kg/ha): "))
p_atual = float(input("Digite o Nível ATUAL de Fósforo (P) no solo (kg/ha): "))
k_atual = float(input("Digite o Nível ATUAL de Potássio (K) no solo (kg/ha): "))

# Preços dos Insumos
print("\n----- Custos de Compra dos Insumos (R$/kg) -----")
custo_N = float(input("Digite o Custo do Nitrogênio (N) [R$/kg]: "))
custo_P = float(input("Digite o Custo do Fósforo (P) [R$/kg]: "))
custo_K = float(input("Digite o Custo do Potássio (K) [R$/kg]: "))

# Parâmetros Agrícolas
A_total = float(input("\nDigite a Área Total disponível para plantio (hectares): "))

# Preço de Venda e Produtividade (Sacos/Hectare)
print("\n----- Produtividade e Preço de Venda (por Saco/Saca) -----")
preco_saco_arroz = float(input("Preço de Venda do Saco de Arroz (R$): "))
sacos_por_ha_arroz = float(input("Produtividade de Arroz (Sacos/ha): "))

preco_saco_feijao = float(input("Preço de Venda do Saco de Feijão (R$): "))
sacos_por_ha_feijao = float(input("Produtividade de Feijão (Sacos/ha): "))

preco_saco_milho = float(input("Preço de Venda do Saco de Milho (R$): "))
sacos_por_ha_milho = float(input("Produtividade de Milho (Sacos/ha): "))

# --- 2. CÁLCULO DE CUSTO VARIÁVEL POR HECTARE (Baseado na deficiência do solo) ---

def calcular_custo_fert_por_ha(cultura, n_atual, p_atual, k_atual, custo_N, custo_P, custo_K):
    """
    Calcula o custo de compra de fertilizantes para 1 hectare,
    somente para repor a deficiência do solo.
    """
    # Define a deficiência de cada nutriente (necessidade - atual). Mínimo é 0.
    deficiencia_N = max(0, NECESSIDADES_IDEAIS[cultura]["N"] - n_atual)
    deficiencia_P = max(0, NECESSIDADES_IDEAIS[cultura]["P"] - p_atual)
    deficiencia_K = max(0, NECESSIDADES_IDEAIS[cultura]["K"] - k_atual)

    # Custo é a deficiência multiplicada pelo preço de compra do insumo
    custo_fert = (
        deficiencia_N * custo_N +
        deficiencia_P * custo_P +
        deficiencia_K * custo_K
    )
    return custo_fert, deficiencia_N, deficiencia_P, deficiencia_K

# Calcular custos por hectare e deficiências para cada cultura
custo_fert_arroz, _, _, _ = calcular_custo_fert_por_ha("Arroz", n_atual, p_atual, k_atual, custo_N, custo_P, custo_K)
custo_fert_feijao, _, _, _ = calcular_custo_fert_por_ha("Feijao", n_atual, p_atual, k_atual, custo_N, custo_P, custo_K)
custo_fert_milho, _, _, _ = calcular_custo_fert_por_ha("Milho", n_atual, p_atual, k_atual, custo_N, custo_P, custo_K)

# --- 3. CÁLCULO DO LUCRO LÍQUIDO POR HECTARE (Coeficiente da Função Objetivo) ---

# Receita por hectare = (Preço de Venda por Saco) * (Sacos por Hectare)
receita_ha_arroz = preco_saco_arroz * sacos_por_ha_arroz
receita_ha_feijao = preco_saco_feijao * sacos_por_ha_feijao
receita_ha_milho = preco_saco_milho * sacos_por_ha_milho

# Lucro Líquido por Hectare = Receita por ha - Custo de Insumos por ha
lucro_liquido_arroz = receita_ha_arroz - custo_fert_arroz
lucro_liquido_feijao = receita_ha_feijao - custo_fert_feijao
lucro_liquido_milho = receita_ha_milho - custo_fert_milho

# --- 4. MODELO DE PROGRAMAÇÃO LINEAR COM PuLP ---

# Criar o modelo (Maximizar Lucro)
model = LpProblem("Planejamento_Agricola_Otimizado", LpMaximize)

# Variáveis de Decisão (Área a ser plantada de cada cultura)
# lowBound=0.5 garante que o programa FORCE o plantio de no mínimo 0.5 ha de cada cultura (Premissa de plantar todas)
# Você pode ajustar o 0.5 para o valor mínimo obrigatório desejado.
x_arroz  = LpVariable("Arroz_ha", lowBound=10)
x_feijao = LpVariable("Feijao_ha", lowBound=10)
x_milho  = LpVariable("Milho_ha", lowBound=10)

# FUNÇÃO OBJETIVO: Maximizar Lucro Líquido Total
model += (
    lucro_liquido_arroz * x_arroz +
    lucro_liquido_feijao * x_feijao +
    lucro_liquido_milho * x_milho
), "Lucro_Total_Liquido"

# RESTRIÇÃO 1: Área Total Disponível
model += x_arroz + x_feijao + x_milho <= A_total, "Area_Total_Max"

# (Não há mais restrições de nutrientes, pois a compra de insumos foi convertida em custo na Função Objetivo)

# --- 5. RESOLUÇÃO E RESULTADOS ---
print("\n--- 🧠 Resolvendo o Modelo de Otimização ---")

# Resolvendo
model.solve()

# Verifica o status da solução
status = LpStatus[model.status]
print(f"Status da Solução: {status}")

if status == "Optimal":
    # Resultados (Área Plantada)
    area_arroz = x_arroz.value()
    area_feijao = x_feijao.value()
    area_milho = x_milho.value()

    # Cálculo dos valores totais para exibição
    lucro_maximo = value(model.objective)
    area_usada = area_arroz + area_feijao + area_milho

    custo_total_insumos = (
        custo_fert_arroz * area_arroz +
        custo_fert_feijao * area_feijao +
        custo_fert_milho * area_milho
    )

    receita_total = (
        receita_ha_arroz * area_arroz +
        receita_ha_feijao * area_feijao +
        receita_ha_milho * area_milho
    )

    print("\n✅ Resultados Otimizados:")
    print("-----------------------------------")
    print("📊 Área plantada ideal:")
    print(f"🌾 Arroz:  {area_arroz:.2f} ha")
    print(f"🫘 Feijão: {area_feijao:.2f} ha")
    print(f"🌽 Milho:  {area_milho:.2f} ha")
    print(f"--- Área Total Usada: {area_usada:.2f}/{A_total:.2f} ha")

    print("\n💰 Análise Financeira (Baseada na Solução):")
    print(f"💲 Receita Bruta Total Estimada: R$ {receita_total:,.2f}")
    print(f"💸 Custo TOTAL de Insumos (Fertilizantes): R$ {custo_total_insumos:,.2f}")
    print(f"📈 Lucro TOTAL Máximo Líquido: R$ {lucro_maximo:,.2f}")

    print("\n⚠️ Nota: Se a deficiência do solo for alta e/ou o custo dos insumos for maior que a receita, o 'Lucro Líquido' de uma cultura pode ser negativo. O modelo tentará minimizar a perda.")

else:
    print("\n❌ O modelo não encontrou uma solução ótima. Verifique as restrições (ex: se a área total é muito pequena para as mínimas obrigatórias).")

--- 📥 Entrada de Dados do Usuário ---
----- Níveis Atuais de Nutrientes no Solo (em kg/ha) -----
Digite o Nível ATUAL de Nitrogênio (N) no solo (kg/ha): 110
Digite o Nível ATUAL de Fósforo (P) no solo (kg/ha): 50
Digite o Nível ATUAL de Potássio (K) no solo (kg/ha): 1

----- Custos de Compra dos Insumos (R$/kg) -----
Digite o Custo do Nitrogênio (N) [R$/kg]: 30
Digite o Custo do Fósforo (P) [R$/kg]: 52
Digite o Custo do Potássio (K) [R$/kg]: 80

Digite a Área Total disponível para plantio (hectares): 100

----- Produtividade e Preço de Venda (por Saco/Saca) -----
Preço de Venda do Saco de Arroz (R$): 130
Produtividade de Arroz (Sacos/ha): 180
Preço de Venda do Saco de Feijão (R$): 196
Produtividade de Feijão (Sacos/ha): 40
Preço de Venda do Saco de Milho (R$): 69
Produtividade de Milho (Sacos/ha): 110

--- 🧠 Resolvendo o Modelo de Otimização ---
Status da Solução: Optimal

✅ Resultados Otimizados:
-----------------------------------
📊 Área plantada ideal:
🌾 Arroz:  80.00 ha
🫘 Feijão: 1